In [ ]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = 'mypassword'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Process Automation for Order Fulfillment

## Description of the use case
This use case involves automating order fulfillment processes using Redis Lua Scripts. By automating these processes, businesses can ensure timely and accurate order processing, improving efficiency and customer satisfaction.

## Redis Data Structures Used
- **Hashes**: To store order and inventory information.
- **Lua Scripts**: To automate order fulfillment processes.


In [ ]:
# Mock data for the use case
orders = {
    'order_1001': {'product_id': 'product_1001', 'quantity': 2, 'status': 'pending'},
    'order_1002': {'product_id': 'product_1002', 'quantity': 4, 'status': 'pending'},
    'order_1003': {'product_id': 'product_1002', 'quantity': 2, 'status': 'pending'},
    'order_1004': {'product_id': 'product_1001', 'quantity': 3, 'status': 'pending'},
    'order_1005': {'product_id': 'product_1002', 'quantity': 1, 'status': 'pending'},
}

inventory = {
    'product_1001': {'name': 'Product A', 'stock': 10},
    'product_1002': {'name': 'Product B', 'stock': 5},
}

# Use Hashes to store order and inventory information
for order_id, data in orders.items():
    r.hset(order_id, mapping=data)

for product_id, data in inventory.items():
    r.hset(product_id, mapping=data)

# Lua Script to automate order fulfillment
fulfill_order_script = '''
local order_id = KEYS[1]
local product_id = redis.call('HGET', order_id, 'product_id')
local quantity = tonumber(redis.call('HGET', order_id, 'quantity'))
local stock = tonumber(redis.call('HGET', product_id, 'stock'))

if stock >= quantity then
    redis.call('HINCRBY', product_id, 'stock', -quantity)
    redis.call('HSET', order_id, 'status', 'fulfilled')
    return 'Order fulfilled'
else
    redis.call('HSET', order_id, 'status', 'out_of_stock')
    return 'Order out of stock'
end
'''

# Function to execute the Lua script for each order
def fulfill_orders():
    for order_id in orders.keys():
        result = r.eval(fulfill_order_script, 1, order_id)
        print(f"{order_id}: {result}")

# Fulfill the orders
fulfill_orders()

# Print the updated order and inventory information
for order_id in orders.keys():
    print(f"{order_id}: {r.hgetall(order_id)}")

for product_id in inventory.keys():
    print(f"{product_id}: {r.hgetall(product_id)}")
